In [7]:
import pandas as pd
import numpy as np
import pickle

import math
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from itertools import chain
from scipy.optimize import curve_fit
import powerlaw

from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers_all-MiniLM-L6-v2')
import oracledb
connection = oracledb.connect(user="", password="", host="",port=1521,service_name="")

from sqlalchemy import create_engine
import cx_Oracle
from tqdm import tqdm
tqdm.pandas()

import itertools

from sentence_transformers.util import cos_sim
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from powerlaw import Fit, plot_pdf

from tqdm import tqdm
import re
import os
from keybert import KeyBERT
kw_model = KeyBERT(model=model)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve,precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# 下载NLTK的punkt数据
# nltk.download('punkt')
stemmer = PorterStemmer()

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

def get_dict(x,dictionary):
    return [dictionary[str(item)] for item in x]
def get_larger(x,number):
    return [item for item in x if item>number]
def compare_larger(x,number):
    return any([item>number for item in x])
def convert_date(x):
    try:
        date=pd.Timestamp(x)
    except:
        date=pd.Timestamp(x.split('-')[0]+'-01-01')
    return date

In [8]:
import itertools
def select_elements(x):
    # 创建一个空列表，用于存储满足条件的元素
    selected_elements = []

    # 获取DataFrame的行数和列数
    n_rows, n_cols = x.shape

    # 遍历每个元素，检查索引是否满足条件
    for i in range(n_rows):
        for j in range(n_cols):
            if j < i:
                selected_elements.append(x.iloc[i, j])
    return selected_elements

In [9]:
def cosine_similarity(list1,list2):
    # Convert lists to numpy arrays
    vector1 = np.array(list1)
    vector2 = np.array(list2)

    # Calculate dot product
    dot_product = np.dot(vector1, vector2)

    # Calculate norms
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Calculate cosine similarity
    cosine_similarity = dot_product / (norm1 * norm2)

    return cosine_similarity

In [10]:
def contains_key_phrase(text, key_phrases):
    for phrase in key_phrases:
        if phrase in str(text):
            return 1
    return 0

# 示例用法
text = "这是一个示例字符串，包含了一些关键词。"
key_phrases = ["示例", "关键词"]
print(contains_key_phrase(text, key_phrases))  # 输出 True


1


In [11]:
sql_string='select t.* from "上市公司匹配patstat_2" t'
df_firm_patstat = pd.read_sql(sql=sql_string, con=connection)
df_firm_patstat

/tmp/ipykernel_101404/3135114047.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_firm_patstat = pd.read_sql(sql=sql_string, con=connection)


,psn_id,psn_name,股票代码,原始企业名称,person_id,appln_id,appln_filing_date,appln_auth,appln_kind
0,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,55537629,2007-12-29 00:00:00,EP,A
1,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,56481707,2007-12-29 00:00:00,EP,A
2,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,406838542,2012-06-18 00:00:00,EP,A
3,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,412387138,2013-04-19 00:00:00,EP,A
4,37556536,ZTE CORPORATION,000063,中兴通讯股份有限公司,4796,414554331,2013-07-26 00:00:00,EP,A
...,...,...,...,...,...,...,...,...,...
2766485,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,568815851,2021-08-20 00:00:00,CN,U
2766486,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569527621,2021-05-21 00:00:00,CN,U
2766487,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569545291,2021-08-26 00:00:00,CN,U
2766488,185438472,"QINGDAO HAIER AIR-CONDITIONING ELECTRONIC CO.,...",600690,海尔智家股份有限公司,85438472,569722490,2021-08-31 00:00:00,CN,U


In [12]:
df_firm_patstat['appln_filing_year']=df_firm_patstat['appln_filing_date'].apply(lambda x:x.year)
dict_appln_filing_year={k:v for k,v in zip(df_firm_patstat['appln_id'],df_firm_patstat['appln_filing_year'])}
dict_appln_date={k:v for k,v in zip(df_firm_patstat['appln_id'],df_firm_patstat['appln_filing_date'])}
dict_appln_date

{55537629: datetime.datetime(2007, 12, 29, 0, 0),
 56481707: datetime.datetime(2007, 12, 29, 0, 0),
 406838542: datetime.datetime(2012, 6, 18, 0, 0),
 412387138: datetime.datetime(2013, 4, 19, 0, 0),
 414554331: datetime.datetime(2013, 7, 26, 0, 0),
 16075341: datetime.datetime(2003, 12, 25, 0, 0),
 328571084: datetime.datetime(2010, 4, 27, 0, 0),
 495792296: datetime.datetime(2017, 12, 8, 0, 0),
 16278451: datetime.datetime(2005, 12, 23, 0, 0),
 56204753: datetime.datetime(2007, 1, 11, 0, 0),
 56264379: datetime.datetime(2008, 2, 8, 0, 0),
 254588: datetime.datetime(2006, 8, 14, 0, 0),
 15868598: datetime.datetime(2001, 6, 13, 0, 0),
 56238657: datetime.datetime(2007, 12, 6, 0, 0),
 16355407: datetime.datetime(2006, 6, 26, 0, 0),
 16348800: datetime.datetime(2006, 3, 29, 0, 0),
 527200941: datetime.datetime(2020, 2, 28, 0, 0),
 16550846: datetime.datetime(1984, 7, 12, 0, 0),
 16644452: datetime.datetime(1986, 12, 8, 0, 0),
 16783416: datetime.datetime(1989, 8, 21, 0, 0),
 17006068: da

In [13]:
sql_string='select t.* from "上市公司匹配patstat_2_cpc" t'
df_firm_patstat_cpc = pd.read_sql(sql=sql_string, con=connection)
df_firm_patstat_cpc

/tmp/ipykernel_101404/395528519.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_firm_patstat_cpc = pd.read_sql(sql=sql_string, con=connection)


,appln_id,cpc_class_symbol
0,2006,H04W 56/0005
1,7927,B60L 7/14
2,7939,B60L2240/12
3,17110,H04L 67/104
4,17110,H04M 1/2535
...,...,...
5324787,575560719,C07C 51/41
5324788,575094138,H01M 50/474
5324789,575345522,H02J 7/00032
5324790,575404974,H04B 10/40


In [113]:
sql_string='select t.* from "上市公司匹配patstat_2_title_abstract" t'
df_firm_patstat_title_abstract = pd.read_sql(sql=sql_string, con=connection)
df_firm_patstat_title_abstract['appln_abstract']=df_firm_patstat_title_abstract['appln_abstract'].progress_apply(lambda x:x.read())
df_firm_patstat_title_abstract


/tmp/ipykernel_101404/4189769416.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_firm_patstat_title_abstract = pd.read_sql(sql=sql_string, con=connection)
100%|███████████████████████████████████████████████████████████████████████| 2382718/2382718 [09:11<00:00, 4324.23it/s]


,appln_id,appln_title,appln_abstract
0,2211,"PYRAZOLOPYRIMIDINONE DERIVATIVES, THEIR PREPAR...",The invention discloses a series of pyrazolopy...
1,1790,METHOD AND DEVICE FOR MONITORING RECEIVING RAD...,The present invention discloses a method and d...
2,1800,A SYSTEM AND METHOD FOR ADJUSTING AMPLITUDE OF...,The present invention discloses a system and m...
3,1815,METHOD AND APPARATUS FOR ELIMINATING NARROW BA...,A device and method for eliminating narrow-ban...
4,2006,A RANDOM ACCESS METHOD FOR USER EQUIPMENT IN T...,The present invention discloses a random acces...
...,...,...,...
2382713,573860416,PAGING MECHANISM FOR WIRELESS COMMUNICATION NE...,"Methods, apparatus, and systems for a paging s..."
2382714,573860530,ARCHITECTURE FOR RANDOM ACCESS MESSAGING,User equipment may request to communicate with...
2382715,573878382,RELEASABLE LOCKING MECHANISM FOR A REFRIGERATE...,A releasable locking mechanism for a refrigera...
2382716,573878426,LINEAR HINGE ASSEMBLY FOR AN APPLIANCE,"A domestic appliance may include a cabinet, a ..."


# AI专利识别

In [116]:
with open('data/list_new_AI_CPC_part1.pickle','rb') as f:
    list_AI_CPC_part1=pickle.load(f)

with open('data/list_new_AI_CPC_part2.pickle','rb') as f:
    list_AI_CPC_part2=pickle.load(f)

list_AI_CPC_part2

['G10L  13/00',
 'G10L  13/02',
 'G10L2013/021',
 'G10L  13/027',
 'G10L  13/033',
 'G10L  13/0335',
 'G10L  13/04',
 'G10L  13/047',
 'G10L  13/06',
 'G10L  13/07',
 'G10L  13/08',
 'G10L2013/083',
 'G10L  13/086',
 'G10L  13/10',
 'G10L2013/105',
 'G10L  25/00',
 'G10L  25/03',
 'G10L  25/06',
 'G10L  25/09',
 'G10L  25/12',
 'G10L  25/15',
 'G10L  25/18',
 'G10L  25/21',
 'G10L  25/24',
 'G10L  25/27',
 'G10L  25/30',
 'G10L  25/33',
 'G10L  25/36',
 'G10L  25/39',
 'G10L  25/45',
 'G10L  25/48',
 'G10L  25/51',
 'G10L  25/54',
 'G10L  25/57',
 'G10L  25/60',
 'G10L  25/63',
 'G10L  25/66',
 'G10L  25/69',
 'G10L  25/72',
 'G10L  25/75',
 'G10L  25/78',
 'G10L2025/783',
 'G10L2025/786',
 'G10L  25/81',
 'G10L  25/84',
 'G10L  25/87',
 'G10L  25/90',
 'G10L2025/903',
 'G10L2025/906',
 'G10L  25/93',
 'G10L2025/932',
 'G10L2025/935',
 'G10L2025/937',
 'G10L  99/00',
 'G11',
 'G11B',
 'G06F  17/14',
 'G06F  17/141',
 'G06F  17/142',
 'G06F  17/144',
 'G06F  17/145',
 'G06F  17/147',
 '

In [134]:
string_part4='artific intelligen, computation intelligen, neural network, neuralnetwork, bayes network, bayesiannetwork, chatbot, data mining, decision model, deep learning, deeplearning, genetic algorithm, inductive logic programm, machine learning, machinelearning, natural language generation, natural language processing, reinforcement learning, supervised learning, supervised training, supervisedlearning, swarm intelligen, swarmintelligen, unsupervised learning, unsupervised training, unsupervisedlearning, semi-supervised learning, semi-supervised training, semisupervised learning, semisupervised training, semi supervisedlearning, semisupervisedlearning, connectionis, expert system, transfer learning, transferlearning, learning algorithm, learning model, support vector machine, random forest, decision tree, gradient tree boosting, xgboost, adaboost, rankboost, logistic regression, stochastic gradient descent, multilayer perceptron, latent semantic analysis, latent dirichlet allocation, multi-agent system, hidden markov model, clustering, combinatorial explosion, comput creativity, deep blue, descriptive model, inductive reasoning, overfitting, predictive analytics, predictive model, target function, test data set, training data set, validation data set, backpropagation, self learning, selflearning, objective function, feature selection, embedding, active learning, regression model, stochastic approach, stochastic technique, stochastic method, stochastic algorithm, probabilist technique, probabilist approach, probabilist method, probabilist algorithm, recommend systemrobot, autonomous system, medical imag, healthcare, virtual assist, personali medic, precision medic, genomic screening, drug discover, medical diagnos, drug creation, medication manag, autonomous vehicle, transportation, driverless, smart car, smart cars, smart city, smart grid, automotive, agriculture, irrigation system, fintech, banking, finance, economics, text analysis, speech analysis, hand writing analysis, handwriting analysis, facial analysis, face, text analytic, speech analytic, hand writing analytic, handwriting analytic, facial analytic, face analytic, text recognition, speech recognition, hand writing recognition, handwriting recognition, facial recognition, face recognition, cybersecurity, predictive analysis, predictive analytic, predictive purchas, marketing analytic, video game'
list_key_phrases=string_part4.split(', ')
list_key_phrases=[i.lower() for i in list_key_phrases]
list_key_phrases

['artific intelligen',
 'computation intelligen',
 'neural network',
 'neuralnetwork',
 'bayes network',
 'bayesiannetwork',
 'chatbot',
 'data mining',
 'decision model',
 'deep learning',
 'deeplearning',
 'genetic algorithm',
 'inductive logic programm',
 'machine learning',
 'machinelearning',
 'natural language generation',
 'natural language processing',
 'reinforcement learning',
 'supervised learning',
 'supervised training',
 'supervisedlearning',
 'swarm intelligen',
 'swarmintelligen',
 'unsupervised learning',
 'unsupervised training',
 'unsupervisedlearning',
 'semi-supervised learning',
 'semi-supervised training',
 'semisupervised learning',
 'semisupervised training',
 'semi supervisedlearning',
 'semisupervisedlearning',
 'connectionis',
 'expert system',
 'transfer learning',
 'transferlearning',
 'learning algorithm',
 'learning model',
 'support vector machine',
 'random forest',
 'decision tree',
 'gradient tree boosting',
 'xgboost',
 'adaboost',
 'rankboost',
 'l

In [135]:
df_firm_patstat_title_abstract['appln_title_AI']=df_firm_patstat_title_abstract['appln_title'].str.lower().progress_apply(lambda x:contains_key_phrase(x,list_key_phrases))
df_firm_patstat_title_abstract['appln_abstract_AI']=df_firm_patstat_title_abstract['appln_abstract'].str.lower().progress_apply(lambda x:contains_key_phrase(x,list_key_phrases))
df_firm_patstat_title_abstract['appln_title_abstract_AI'] = np.where(
    (df_firm_patstat_title_abstract['appln_title_AI'] == 1) | (df_firm_patstat_title_abstract['appln_abstract_AI'] == 1),
    1, 0
)
df_firm_patstat_title_abstract

100%|██████████████████████████████████████████████████████████████████████| 2382718/2382718 [02:53<00:00, 13726.63it/s]


,appln_id,appln_title,appln_abstract,appln_title_AI,appln_abstract_AI,appln_title_abstract_AI
0,2211,"PYRAZOLOPYRIMIDINONE DERIVATIVES, THEIR PREPAR...",The invention discloses a series of pyrazolopy...,0,0,0
1,1790,METHOD AND DEVICE FOR MONITORING RECEIVING RAD...,The present invention discloses a method and d...,0,0,0
2,1800,A SYSTEM AND METHOD FOR ADJUSTING AMPLITUDE OF...,The present invention discloses a system and m...,0,0,0
3,1815,METHOD AND APPARATUS FOR ELIMINATING NARROW BA...,A device and method for eliminating narrow-ban...,0,0,0
4,2006,A RANDOM ACCESS METHOD FOR USER EQUIPMENT IN T...,The present invention discloses a random acces...,0,0,0
...,...,...,...,...,...,...
2382713,573860416,PAGING MECHANISM FOR WIRELESS COMMUNICATION NE...,"Methods, apparatus, and systems for a paging s...",0,0,0
2382714,573860530,ARCHITECTURE FOR RANDOM ACCESS MESSAGING,User equipment may request to communicate with...,0,0,0
2382715,573878382,RELEASABLE LOCKING MECHANISM FOR A REFRIGERATE...,A releasable locking mechanism for a refrigera...,0,0,0
2382716,573878426,LINEAR HINGE ASSEMBLY FOR AN APPLIANCE,"A domestic appliance may include a cabinet, a ...",0,0,0


In [136]:
df_firm_patstat_cpc['AI_CPC_part1']=np.where(df_firm_patstat_cpc['cpc_class_symbol'].isin(list_AI_CPC_part1),1,0)
df_firm_patstat_cpc['AI_CPC_part2']=np.where(df_firm_patstat_cpc['cpc_class_symbol'].isin(list_AI_CPC_part2),1,0)
df_firm_patstat_cpc

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC
0,2006,H04W 56/0005,0,0,0,0
1,7927,B60L 7/14,0,0,0,0
2,7939,B60L2240/12,0,0,0,0
3,17110,H04L 67/104,0,0,0,0
4,17110,H04M 1/2535,0,0,0,0
...,...,...,...,...,...,...
5324787,575560719,C07C 51/41,0,0,0,0
5324788,575094138,H01M 50/474,0,0,0,0
5324789,575345522,H02J 7/00032,0,0,0,0
5324790,575404974,H04B 10/40,0,0,0,0


In [137]:
df_firm_patstat_cpc['block2']=np.where(df_firm_patstat_cpc['appln_id'].isin(
    df_firm_patstat_title_abstract[df_firm_patstat_title_abstract['appln_title_abstract_AI']==1]['appln_id']
),1,0)
df_firm_patstat_cpc

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC
0,2006,H04W 56/0005,0,0,0,0
1,7927,B60L 7/14,0,0,0,0
2,7939,B60L2240/12,0,0,0,0
3,17110,H04L 67/104,0,0,0,0
4,17110,H04M 1/2535,0,0,0,0
...,...,...,...,...,...,...
5324787,575560719,C07C 51/41,0,0,0,0
5324788,575094138,H01M 50/474,0,0,1,0
5324789,575345522,H02J 7/00032,0,0,0,0
5324790,575404974,H04B 10/40,0,0,1,0


In [138]:
df_firm_patstat_cpc['AI_CPC']=np.where((df_firm_patstat_cpc['AI_CPC_part1']==1)|((df_firm_patstat_cpc['AI_CPC_part2']==1)&(df_firm_patstat_cpc['block2']==1)),1,0)
df_firm_patstat_cpc

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC
0,2006,H04W 56/0005,0,0,0,0
1,7927,B60L 7/14,0,0,0,0
2,7939,B60L2240/12,0,0,0,0
3,17110,H04L 67/104,0,0,0,0
4,17110,H04M 1/2535,0,0,0,0
...,...,...,...,...,...,...
5324787,575560719,C07C 51/41,0,0,0,0
5324788,575094138,H01M 50/474,0,0,1,0
5324789,575345522,H02J 7/00032,0,0,0,0
5324790,575404974,H04B 10/40,0,0,1,0


In [139]:
len(df_firm_patstat_cpc[(df_firm_patstat_cpc['AI_CPC']==1)]['appln_id'].unique())

43665

In [140]:
df_firm_patstat_cpc_AI = df_firm_patstat_cpc.groupby('appln_id').filter(lambda x: (x['AI_CPC'] > 0).any())
df_firm_patstat_cpc_AI

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC
380,7777571,H04N 21/4586,0,0,0,0
479,8143719,H04N 21/26258,0,0,0,0
486,8142877,H04N 7/17327,0,0,0,0
487,8142877,H04N 19/115,0,0,0,0
488,8142877,H04N 19/625,0,0,0,0
...,...,...,...,...,...,...
5324586,572736901,G06F 16/29,0,0,1,0
5324609,573464069,G06F2203/011,0,0,1,0
5324657,573498187,G06T2207/20081,1,0,0,1
5324761,575542038,G06N 3/08,1,0,1,1


In [141]:
df_firm_patstat_cpc_AI=df_firm_patstat_cpc_AI.sort_values(['appln_id'])
df_firm_patstat_cpc_AI['appln_date']=df_firm_patstat_cpc_AI['appln_id'].map(dict_appln_date)
df_firm_patstat_cpc_AI

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC,appln_date
4576095,24599,H01M 8/04097,0,0,0,0,2005-12-30
4243158,24599,H01M 8/04,0,0,0,0,2005-12-30
4992327,24599,H01M 8/22,0,0,0,0,2005-12-30
998957,24599,H01M 8/06,1,0,0,1,2005-12-30
1163818,24599,Y02P 70/50,0,0,0,0,2005-12-30
...,...,...,...,...,...,...,...
2496512,575553198,G05D 1/0219,1,0,0,1,2021-12-27
333345,575553198,G05D2201/0208,0,0,0,0,2021-12-27
1914269,575554198,G10L 15/32,1,0,1,1,2021-12-31
4909513,575554198,G10L 15/02,1,0,1,1,2021-12-31


In [142]:
df_firm_patstat_cpc_AI['appln_filing_year']=df_firm_patstat_cpc_AI['appln_date'].apply(lambda x:x.year)
df_firm_patstat_cpc_AI

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC,appln_date,appln_filing_year
4576095,24599,H01M 8/04097,0,0,0,0,2005-12-30,2005
4243158,24599,H01M 8/04,0,0,0,0,2005-12-30,2005
4992327,24599,H01M 8/22,0,0,0,0,2005-12-30,2005
998957,24599,H01M 8/06,1,0,0,1,2005-12-30,2005
1163818,24599,Y02P 70/50,0,0,0,0,2005-12-30,2005
...,...,...,...,...,...,...,...,...
2496512,575553198,G05D 1/0219,1,0,0,1,2021-12-27,2021
333345,575553198,G05D2201/0208,0,0,0,0,2021-12-27,2021
1914269,575554198,G10L 15/32,1,0,1,1,2021-12-31,2021
4909513,575554198,G10L 15/02,1,0,1,1,2021-12-31,2021


In [143]:
dict_AI_str={0:'No-AI',1:'AI'}
df_firm_patstat_cpc_AI['cpc_class_symbol_AI']=df_firm_patstat_cpc_AI['cpc_class_symbol']+'_'+df_firm_patstat_cpc_AI['AI_CPC'].map(dict_AI_str)
df_firm_patstat_cpc_AI

,appln_id,cpc_class_symbol,AI_CPC_part1,AI_CPC_part2,block2,AI_CPC,appln_date,appln_filing_year,cpc_class_symbol_AI
4576095,24599,H01M 8/04097,0,0,0,0,2005-12-30,2005,H01M 8/04097_No-AI
4243158,24599,H01M 8/04,0,0,0,0,2005-12-30,2005,H01M 8/04_No-AI
4992327,24599,H01M 8/22,0,0,0,0,2005-12-30,2005,H01M 8/22_No-AI
998957,24599,H01M 8/06,1,0,0,1,2005-12-30,2005,H01M 8/06_AI
1163818,24599,Y02P 70/50,0,0,0,0,2005-12-30,2005,Y02P 70/50_No-AI
...,...,...,...,...,...,...,...,...,...
2496512,575553198,G05D 1/0219,1,0,0,1,2021-12-27,2021,G05D 1/0219_AI
333345,575553198,G05D2201/0208,0,0,0,0,2021-12-27,2021,G05D2201/0208_No-AI
1914269,575554198,G10L 15/32,1,0,1,1,2021-12-31,2021,G10L 15/32_AI
4909513,575554198,G10L 15/02,1,0,1,1,2021-12-31,2021,G10L 15/02_AI


In [144]:
df_firm_patstat_cpc_AI.to_pickle('data/df_firm_patstat_cpc_AI.pickle')

# 计算替代性/互补性

In [145]:
filing_year_list=list(range(1995,2023))
# filing_year_list=2001
dict_year_pairs={}
for filing_year in filing_year_list:
    df_item=df_firm_patstat_cpc_AI[df_firm_patstat_cpc_AI['appln_filing_year']==filing_year]
    pair_list=[]
    total_cpc_list=df_item['cpc_class_symbol_AI'].unique()
    for appln_id in tqdm(df_item['appln_id'].unique()):
        cpc_4_list=df_item[df_item['appln_id']==appln_id]['cpc_class_symbol_AI'].unique()
        cpc_4_pair=list(itertools.combinations(cpc_4_list, 2))
        pair_list.append(cpc_4_pair)
    year_pairs=list(chain(*pair_list))
    dict_year_pairs[filing_year]=year_pairs

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 2139.77it/s]


In [ ]:
filing_year_list=list(range(2000,2023))
for filing_year in tqdm(filing_year_list):
    print(filing_year,'----------------------------')
    df_item=df_firm_patstat_cpc_AI[df_firm_patstat_cpc_AI['appln_filing_year'].isin(range(filing_year-2,filing_year+1))]
    total_cpc_list=df_item['cpc_class_symbol_AI'].unique()
    df_matrix=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    df_sub=pd.DataFrame(columns=total_cpc_list,index=total_cpc_list)
    
    dict_pair_count=Counter(chain(*[dict_year_pairs[year] for year in range(filing_year-2,filing_year+1)]))
    for k in dict_pair_count.keys():
        df_matrix.loc[k[0],k[1]]=dict_pair_count[k]
        df_matrix.loc[k[1],k[0]]=dict_pair_count[k]
    df_matrix.fillna(0,inplace=True)
    df_matrix.to_pickle('matrix/{}_matrix_3.pickle'.format(str(filing_year)))
    selected_elements=select_elements(df_matrix)

    mean=np.mean(selected_elements)
    std=np.std(selected_elements)
    df_matrix_st=(df_matrix-mean)/std

    selected_elements_st=select_elements(df_matrix_st)
    min_value=np.min(selected_elements_st)
    max_value=np.max(selected_elements_st)
    df_matrix_mm=(df_matrix_st-min_value)/ (max_value - min_value)
    df_matrix_mm.to_pickle('matrix/{}_matrix_mm_3.pickle'.format(str(filing_year)))
    
    # 获取DataFrame的行数和列数
    n_rows, n_cols = df_sub.shape

    # 遍历每个元素，检查索引是否满足条件
    for i in range(n_rows):
        for j in range(n_cols):
            if j < i:
                cosine_similarity_i_j=cosine_similarity(df_matrix_mm.iloc[:, i],df_matrix_mm.iloc[:, j])
                df_sub.iloc[i,j]=cosine_similarity_i_j
                df_sub.iloc[j,i]=cosine_similarity_i_j
    df_sub.to_pickle('matrix/{}_matrix_sub_3.pickle'.format(str(filing_year)))


  0%|                                                                                            | 0/23 [00:00<?, ?it/s]

2000 ----------------------------



  4%|███▋                                                                                | 1/23 [00:20<07:33, 20.61s/it]

2001 ----------------------------



  9%|███████▎                                                                            | 2/23 [00:21<03:03,  8.72s/it]

...

/tmp/ipykernel_101404/3711813477.py:14: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = dot_product / (norm1 * norm2)
 74%|█████████████████████████████████████████████████████████▋                    | 17/23 [1:00:35<1:26:00, 860.09s/it]

2017 ----------------------------


/tmp/ipykernel_101404/3711813477.py:14: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = dot_product / (norm1 * norm2)
 78%|████████████████████████████████████████████████████████████▎                | 18/23 [2:01:28<2:21:36, 1699.24s/it]

2018 ----------------------------


/tmp/ipykernel_101404/3711813477.py:14: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = dot_product / (norm1 * norm2)


# 计算互补和替代

In [ ]:
df_firm_patstat_cpc_AI_group=df_firm_patstat_cpc_AI.pivot_table(index=['appln_id'], columns='AI_CPC', 
                                                                      values='cpc_class_symbol_AI', aggfunc=list).reset_index().rename({0:'No-AI',1:'AI'},axis=1).reset_index(drop=True)
df_firm_patstat_cpc_AI_group['No-AI'] = df_firm_patstat_cpc_AI_group['No-AI'].apply(lambda d: d if isinstance(d, list) else [])

df_firm_patstat_cpc_AI_group

In [ ]:
dict_matrix_mms=dict()
for i in tqdm([i for i in os.listdir('matrix/') if '_matrix_mm_3' in i]):
    dict_matrix_mms[int(i[0:4])]=pd.read_pickle('matrix/'+i)

dict_matrix_subs=dict()
for i in tqdm([i for i in os.listdir('matrix/') if '_matrix_sub_3' in i]):
    dict_matrix_subs[int(i[0:4])]=pd.read_pickle('matrix/'+i)

In [ ]:
for index in tqdm(df_firm_patstat_cpc_AI_group.index):
    appln_id=df_firm_patstat_cpc_AI_group.loc[index,'appln_id']
    non_ai_list=df_firm_patstat_cpc_AI_group.loc[index,'No-AI']
    ai_list=df_firm_patstat_cpc_AI_group.loc[index,'AI']
    if dict_appln_filing_year[appln_id] in range(2000,2023) and len(non_ai_list)>0 and len(ai_list)>0:
#         non_ai_list=list(set([i for i in non_ai_list]))
#         ai_list=list(set([i for i in ai_list]))
        #删掉重复的CPC。因为重复不代表交叉，不存在意义
        list_ai_pairs=[[i,j] for i in non_ai_list for j in ai_list if i != j]

        df_firm_patstat_cpc_AI_group.loc[index,'complementarity_mean']=np.mean(
            [dict_matrix_mms[dict_appln_filing_year[appln_id]].loc[list_ai_pair[0],list_ai_pair[1]] for list_ai_pair in list_ai_pairs]
        )
        df_firm_patstat_cpc_AI_group.loc[index,'substitution_mean']=np.mean(
            [dict_matrix_subs[dict_appln_filing_year[appln_id]].loc[list_ai_pair[0],list_ai_pair[1]] for list_ai_pair in list_ai_pairs]
        )
        df_firm_patstat_cpc_AI_group.loc[index,'complementarity_sum']=np.sum(
            [dict_matrix_mms[dict_appln_filing_year[appln_id]].loc[list_ai_pair[0],list_ai_pair[1]] for list_ai_pair in list_ai_pairs]
        )
        df_firm_patstat_cpc_AI_group.loc[index,'substitution_sum']=np.sum(
            [dict_matrix_subs[dict_appln_filing_year[appln_id]].loc[list_ai_pair[0],list_ai_pair[1]] for list_ai_pair in list_ai_pairs]
        )
    if dict_appln_filing_year[appln_id] in range(2000,2023) and len(non_ai_list)==0:
        df_firm_patstat_cpc_AI_group.loc[index,'complementarity_mean']=0
        df_firm_patstat_cpc_AI_group.loc[index,'substitution_mean']=1
        df_firm_patstat_cpc_AI_group.loc[index,'complementarity_sum']=0
        df_firm_patstat_cpc_AI_group.loc[index,'substitution_sum']=1
df_firm_patstat_cpc_AI_group

In [ ]:
df_firm_patstat_cpc_AI_group.to_pickle('data/df_firm_patstat_cpc_AI_group.pickle')